### 前言

  作为一个刚从学校走出来的新人，每年最为关心的就是高校排名。目前国内外都有几个比较主流的平台，比如美国U.S. News、
英国QS；在国内软科世界大学排名、中国台湾大学（NTU）版世界大学排名也具有一定的影响力。此次选择QS发布的2022年世界高校排名数据作为研究对象，一窥世界最知名高校排行榜。

QS的排名是依据以下6个指标对各大学进行评估：

学术声誉（40%）

雇主声誉（10%）

师生比（20%）

每名教师的引用率（20%）

国际教师比例（5%）

留学生比例（5%）
因此我们需要在QS官网获取到大学排名相关数据以及以下几个指标的数据。

通过一个简单的爬虫就可以实现这个小目标。代码如下：

In [17]:
import pandas as pd
import numpy as np
import requests
import bs4
import re
import json
from bs4 import BeautifulSoup

In [18]:
def askURL(url):
    head = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}
    
    try:
        response = requests.get(url,headers=head)
        response.encoding = response.apparent_encoding
        print(response.status_code)
    except Exception as e:
        print(f'爬取失败,失败的元因:{e}')
    return(response.text)       

In [19]:
def Resolve_1(columns):
    Lst=[]
    for i in range(len(result['data'])):
        m = result['data'][i][columns]
        Lst.append(m)
    return(Lst)

In [20]:
def Rssolve_2(s,columns):
    Lst = []
    for i in range(len(result['data'])):
        m = re.findall(s,result['data'][i][columns])[0]
        Lst.append(m)
    return(Lst)

In [21]:
url = 'https://www.qschina.cn/sites/default/files/qs-rankings-data/cn/2122636_indicators.txt
Request Method: GE'
html = askURL(url)
result = json.loads(html)
country =Resolve_1('location')
overall_rank = Resolve_1('overall_rank')
University = Rssolve_2(r'class="uni-link">(.*?)<','uni')
overall = Rssolve_2(r'class="td-wrap-in">(.*?)<','overall')
ISR = Rssolve_2(r'class="td-wrap-in">(.*?)<','ind_14')
IFR = Rssolve_2(r'class="td-wrap-in">(.*?)<','ind_18')
FSR = Rssolve_2(r'class="td-wrap-in">(.*?)<','ind_36')
CpF = Rssolve_2(r'class="td-wrap-in">(.*?)<','ind_73')
AR = Rssolve_2(r'class="td-wrap-in">(.*?)<','ind_76')
ER = Rssolve_2(r'class="td-wrap-in">(.*?)<','ind_77')

200


KeyError: 'location'

将获取数据存为DataFrame类型

In [7]:
df_rank = pd.DataFrame(
    {'学校':University,'国家':country,'排名':overall_rank,'总分':overall,'国际学生比例':ISR,
     '国际教师比例':IFR,'师生比例':FSR,'每个学院的引文':CpF,'学术声誉':AR,'雇主声誉':ER})

把最终数据以Excel格式保存到本地文件中。

In [8]:
df_rank.to_excel(r'G:\数据集\2022QS\qs_university_rank.xlsx',index=False)

既然已经获取了数据就让我们来一窥究竟吧！

In [9]:
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.components import Table
import re
import numpy as np
import pandas as pd

In [30]:
# 读取数据
data = pd.read_excel(r'G:\数据集\2022QS\qs_university_rank.xlsx',encoding='utf8')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 10 columns):
学校         1300 non-null object
国家         1299 non-null object
排名         1300 non-null int64
总分         501 non-null float64
国际学生比例     1275 non-null float64
国际教师比例     1228 non-null float64
师生比例       1299 non-null float64
每个学院的引文    1300 non-null float64
学术声誉       1300 non-null float64
雇主声誉       1300 non-null float64
dtypes: float64(7), int64(1), object(2)
memory usage: 101.6+ KB


In [31]:
# 由于榜单将中国大陆、香港特区、澳门特区、台湾省等地的院校分开记录，先将他们统一归为中国(China)
data['loc'] = data['国家']
data['国家'].replace(['China (Mainland)','Hong Kong SAR','Macau SAR','Taiwan'],'China',inplace=True)

In [32]:
tool_js = """
<div style="border-bottom: 1px solid rgba(255,255,255,.3); font-size: 18px;padding-bottom: 7px;margin-bottom: 7px">
                 {} 
                 </div>
                 排名：{} <br>
                 国家地区：{} <br>
                 加权总分：{} <br>
                 国际学生比例：{} <br>
                 国际教师比例：{} <br>
                 师生比例：{} <br>
                 学术声誉：{} <br>
                 雇主声誉：{} <br>
                 教员引用率：{} <br>
"""

我们筛选一下排名前100的世界级高校，看看这些学校在世界各国的分布情况。

In [33]:
# 将top100的大学按照总分从低到高排序

top_100_data = data[data['排名']<=100]
top_100_data = top_100_data.sort_values(by='总分',ascending=True)

In [34]:
university,score = [],[]
for idx,row in top_100_data.iterrows():
    tjs = tool_js.format(row['学校'],row['排名'],row['国家'],row['总分'],row['国际学生比例'],row['国际教师比例'],row['师生比例'],row['学术声誉'],row['雇主声誉'],row['每个学院的引文'])  
    if row['国家'] == 'China':
        university.append('CN {}'.format(re.sub('(.*?)','',row['国家'])))
    else:
        university.append(re.sub('(.*?)','',row['国家']))
    score.append(opts.BarItem(name='',value=row['总分'],tooltip_opts=opts.TooltipOpts(formatter=tjs)))

In [35]:
bar = (Bar()
       .add_xaxis(university)
       .add_yaxis('',score,category_gap='30%')
       .set_global_opts(title_opts=opts.TitleOpts(title='QS2022年世界大学排名top100',
                                                  pos_left='center',
                                                  title_textstyle_opts=opts.TextStyleOpts(font_size=20)),
                       datazoom_opts=opts.DataZoomOpts(range_start=70,range_end=100,orient='vertical'),
                       visualmap_opts=opts.VisualMapOpts(is_show=False,max_=100,min_=60,dimension=0,
                                                         range_color=['#008B8B', '#FF7F50']),
                       legend_opts=opts.LegendOpts(is_show=False),
                       xaxis_opts=opts.AxisOpts(is_show=False,is_scale=True),
                       yaxis_opts=opts.AxisOpts(axistick_opts=opts.AxisTickOpts(is_show=False),
                                                 axisline_opts=opts.AxisLineOpts(is_show=False),
                                                 axislabel_opts=opts.LabelOpts(font_size=12)))
       .set_series_opts(label_opts=opts.LabelOpts(is_show=True,position='right',font_style='italic'),
                        itemstyle_opts={'normal':{"barBorderRadius": [30, 30, 30, 30],
                                                    'shadowBlur': 10,
                                                    'shadowColor': 'rgba(120, 36, 50, 0.5)',
                                                    'shadowOffsetY': 5,}}).reversal_axis())

In [36]:
grid = (
        Grid(init_opts=opts.InitOpts(theme='purple-passion', width='1000px', height='1200px'))
        .add(bar, grid_opts=opts.GridOpts(pos_right='10%', pos_left='20%'))
    )
grid.render_notebook()

1.排名第一的是麻省理工学院，除去国际学生比例这一项，其余均为100分。
2.排名前五的分别为：麻省理工学院、牛津大学、斯坦福大学、剑桥大学、哈弗大学，都是历史悠久的世界名校。
3.可以看到国内排名最高的是清华大学，位列17位，紧随其后的是北京大学第18位。

In [12]:
top_100_data = data[data['排名']<=100]
top_100_data = top_100_data.groupby(['国家'])['学校'].count().reset_index()
top_100_data.columns = ['国家', '数量']
top_100_data = top_100_data.sort_values(by='数量' , ascending=False) 


bar = (Bar(init_opts=opts.InitOpts(theme='purple-passion', width='1000px', height='600px'))
       .add_xaxis(top_100_data['国家'].tolist())
       .add_yaxis('拥有名校数量', top_100_data['数量'].tolist(), category_gap='50%')
       .set_global_opts(title_opts=opts.TitleOpts(title="top100高校分布情况",
                                                  pos_left="center",
                                                  title_textstyle_opts=opts.TextStyleOpts(font_size=20)),
                        visualmap_opts=opts.VisualMapOpts(is_show=False, max_=300, min_=0, dimension=1,
                                range_color=['#00FFFF', '#FF7F50']),
                        legend_opts=opts.LegendOpts(is_show=False),
                        xaxis_opts=opts.AxisOpts(axistick_opts=opts.AxisTickOpts(is_show=False),
                                                 axisline_opts=opts.AxisLineOpts(is_show=False),
                                                 axislabel_opts=opts.LabelOpts(font_size=15)),
                        yaxis_opts=opts.AxisOpts(is_show=False))
       .set_series_opts(label_opts=opts.LabelOpts(is_show=True,
                                                  position='top',
                                                  font_size=15,
                                                  font_style='italic'),
                        itemstyle_opts={"normal": {
                                                    "barBorderRadius": [30, 30, 30, 30],
                                                    'shadowBlur': 10,
                                                    'shadowColor': 'rgba(120, 36, 50, 0.5)',
                                                    'shadowOffsetY': 5,
                                                }
                                       }
))

bar.render_notebook()

1.排名前100的世界名校中美国拥有27所，位居第一；英国拥有17所，位居第二。
2.中国各大院校排进世界名校前100的共有12所，排名第三。
3.可以看出拥有众多名校是美国和英国每年能够吸引一众国内优秀的人才前往留学的重要原因。
4.同时拥有众多优秀的世界级名校也是这些国家在科研学术领域取得领先地位的坚实基础。

In [13]:
# top500名校在各国分布情况
top_500_data = data[data['排名']<=500]
top_500_data = top_500_data.groupby(['国家'])['学校'].count().reset_index()
top_500_data.columns = ['国家', '数量']
top_500_data = top_500_data.sort_values(by='数量' , ascending=False)  

In [14]:
# top500名校最多的国家前十位在地图上显示出来
country_list = ['China', 'United States', 'United Kingdom','Australia','Korea', 'Japan', 'Russia', 'Canada','Germany','Italy']
# 对应国家的经纬度
loc = {
    'Australia': [133.867328,-23.851273],
    'Canada': [-106.346771, 56.130366],
    'Korea':[127.727805,36.780596],
    'Japan':[139.717668,35.75119], 
    'Russia':[85.708379,63.631202],
    'China': [104.195397, 35.86166],
    'Germany': [13.328239,52.599408],
    'United Kingdom': [-3.435973, 55.378051],
    'United States': [-95.712891, 37.09024],
    'Italy': [11.751246,43.746732],
}

In [15]:
data_pair = []
for idx, row in top_500_data.iterrows():
    if row['国家'] == 'China':
        data_pair.append(['China', row['数量']])
    else:
        data_pair.append([row['国家'], row['数量']])

In [16]:
fmt_js = """function (params) {return params.name+': '+Number(params.value[2]);}"""

mp = Map()
mp.add(
        "高校数量",
        data_pair,
        "world",
        is_map_symbol_show=False,
        is_roam=False)

mp.set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                          itemstyle_opts={'normal': {
                                                'areaColor': '#40E0D0',
                                                'borderColor': 'k',
                                                'shadowColor': '#FFD700',
                                                'shadowBlur': 20,
                                                'opacity': 0.8
                                                    }
                                        })
    
mp.set_global_opts(
        title_opts=opts.TitleOpts(title="top500高校按国家分布", pos_left='center',
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=18)),
        legend_opts=opts.LegendOpts(is_show=False),
        visualmap_opts=opts.VisualMapOpts(is_show=False, 
                                          max_=100,
                                          is_piecewise=False,
                                          dimension=0,
                                          range_color=['rgba(255,228,225,0.6)', 'rgba(255,0,0,0.9)', 'rgba(255,0,0,1)'])
    )
data_pair = [[x, y] for x, y in data_pair if x in country_list]    
geo = Geo()

In [17]:
# 需要先将几个国家的经纬度信息加入到geo中
for k, v in loc.items():
    geo.add_coordinate(k, v[0], v[1])
# 这里将geo的地图透明度配置为0
geo.add_schema(maptype="world", is_roam=False, itemstyle_opts={'normal': {'opacity': 0}})
    
geo.add("", data_pair, symbol_size=1)
# 显示标签配置
geo.set_series_opts(
    label_opts=opts.LabelOpts(
            is_show=True,
            position='right',
            color='white',
            font_size=12,
            font_weight='bold',
            formatter=JsCode(fmt_js)),
    )
    
grid = (
        Grid(init_opts=opts.InitOpts(theme='chalk', width='1000px', height='600px'))
        .add(mp, grid_opts=opts.GridOpts(pos_top="12%"))
        .add(geo, grid_opts=opts.GridOpts(pos_bottom="12%"))
    )

grid.render_notebook()


1.可以看到占据世界排名前500名校最多的10个国家几乎都分布在欧洲、北美洲和东亚地区，都是经济相对发达，社会安定的地区。
2.前500的名校中美国拥有87所占据榜首。
3.中国有43所高校上榜，位居第三。
4.可以看出近年来我国在教育方面投入巨大，尤其是高等教育，也取得了一定的进步，越来越多的国内优秀高校国际认可，这对我国留住本国人才，吸引更多外来人才，大力发展科技，提高科研能力提供了良好的土壤。

### 对比中美英三国名校情况

In [18]:
# 散点图1展示中国，美国，英国各大学校师资力量情况

cn_data_x, cn_data_y = [], []
for idx, row in data[(data['国家']=='China') & (data['排名']<=1000)].iterrows():
    cn_data_y.append([row['每个学院的引文'], row['学术声誉'], row['排名'], row['学校']])
    cn_data_x.append(row['师生比例'])
    
un_data_x, un_data_y = [], []
for idx, row in data[(data['国家']=='United States') & (data['排名']<=1000)].iterrows():
    un_data_y.append([row['每个学院的引文'], row['学术声誉'], row['排名'], row['学校']])
    un_data_x.append(row['师生比例'])
    
uk_data_x, uk_data_y = [], []
for idx, row in data[(data['国家']=='United Kingdom') & (data['排名']<=1000)].iterrows():
    uk_data_y.append([row['每个学院的引文'], row['学术声誉'], row['排名'], row['学校']])
    uk_data_x.append(row['师生比例'])

In [19]:
tool_js = """
function (obj) {
            var value = obj.value;
            var schema = [{name: '排名', index: 0, text: '总体排名'},
                            {name: '学术声誉', index: 1, text: '学术声誉'},
                            {name: '每个学院的引文', index: 2, text: '每个学院的引文'},
                            {name: '师生比例', index: 3, text: '师生比例'}];
            return '<div style="border-bottom: 1px solid rgba(255,255,255,.3); font-size: 18px;padding-bottom: 7px;margin-bottom: 7px">'
                + value[4] 
                + '</div>'
                + schema[0].text + '：' + value[3] + '<br>'
                + schema[1].text + '：' + value[2] + '<br>'
                + schema[2].text + '：' + value[1] + '<br>'
                + schema[3].text + '：' + value[0] + '<br>';
        }
"""

In [20]:
# 设置风格
itemStyle_cn = {
    'opacity': 0.8,
    'shadowBlur': 10,
    'shadowOffsetX': 0,
    'shadowOffsetY': 0,
    'shadowColor': 'rgba(0, 0, 0, 0.5)',
    'color': '#dd4444'
}

itemStyle_un = {
    'opacity': 0.8,
    'shadowBlur': 10,
    'shadowOffsetX': 0,
    'shadowOffsetY': 0,
    'shadowColor': 'rgba(0, 0, 0, 0.5)',
    'color': '#4B0082'
}

itemStyle_uk = {
    'opacity': 0.8,
    'shadowBlur': 10,
    'shadowOffsetX': 0,
    'shadowOffsetY': 0,
    'shadowColor': 'rgba(0, 0, 0, 0.5)',
    'color': '#0000FF'
}


In [21]:
visualMap = [{'left': 'right','top': '10%','dimension': 2,'min': 20,'max': 100,'itemWidth': 30,
              'itemHeight': 120,'calculable': True,'precision': 0.1,'text': ['圆形大小：学术声誉'],'textGap': 30,
              'textStyle': {'color': '#fff'},'inRange': {'symbolSize': [10, 50]},
              'outOfRange': {'symbolSize': [10, 50],'color': ['rgba(255,255,255,.2)']},
              'controller': {'inRange': {'color': ['#c23531']},'outOfRange': {'color': ['#444']}}},
             {'left': 'right','bottom': '15%','dimension': 3,'min': 1,'max': 500,'itemHeight': 120,
              'precision': 0.1,'text': ['明暗：总体排名'],'textGap': 30,'textStyle': {'color': '#fff'},
              'inRange': {'colorLightness': [0.5, 1]},
              'outOfRange': {'color': ['rgba(255,255,255,.2)']},
              'controller': {'inRange': {'color': ['#c23531']},'outOfRange': {'color': ['#444']}}}]


In [22]:
scatter_cn = (Scatter(init_opts=opts.InitOpts(bg_color='#404a59',width='1000px', height='800px'))
            .add_xaxis(cn_data_x)
            .add_yaxis("中国", cn_data_y,
                       label_opts=opts.LabelOpts(is_show=False),
                       itemstyle_opts=itemStyle_cn)
            .set_global_opts(yaxis_opts=opts.AxisOpts(name='教员引用率', type_="value", is_scale=True, 
                                                      name_textstyle_opts={'color': '#fff', 'fontSize': 16},
                                                      axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#eee'))),
                             xaxis_opts=opts.AxisOpts(name='师生比', type_="value", is_scale=True, 
                                                      name_textstyle_opts={'color': '#fff', 'fontSize': 16},
                                                      axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#eee'))),
                             tooltip_opts=opts.TooltipOpts(is_show=True, background_color='#222', border_color='#777', border_width=1, formatter=JsCode(tool_js)),
                             legend_opts=opts.LegendOpts(is_show=True, pos_right=10,
                                                         textstyle_opts=opts.TextStyleOpts(color='#fff',font_size=16)),
                             visualmap_opts=visualMap))

scatter_un = (Scatter(init_opts=opts.InitOpts(bg_color='#404a59',width='1000px', height='800px'))
            .add_xaxis(un_data_x)
            .add_yaxis("美国", un_data_y,
                       label_opts=opts.LabelOpts(is_show=False),
                       itemstyle_opts=itemStyle_un
                       )
            )
            
scatter_uk = (Scatter(init_opts=opts.InitOpts(bg_color='#404a59',width='1000px', height='800px'))
            .add_xaxis(uk_data_x)
            .add_yaxis("英国", uk_data_y,
                       label_opts=opts.LabelOpts(is_show=False),
                       itemstyle_opts=itemStyle_uk
                       )
            )
            
scatter_cn.overlap(scatter_un)
scatter_cn.overlap(scatter_uk)
scatter_cn.render_notebook()

图中：x轴为师生比，y轴为教员引用率，点的大小反映学校声誉，点的明暗反映总排名。
1.可以看出在师资力量对比上中美英三国高校都比较接近，美国和英国表现会更加突出一些。
2.右上角被美国的高校完全占据，这也反映了我国与美国对高级人才的把控与培养方面还具有差距，期望国内能够给予科研人员，高校教授更优质的待遇，以期吸引更多人才。

In [23]:
# 散点图2展示中国，美国，英国各大学校国际化程度对比情况

cn_data_x, cn_data_y = [], []
for idx, row in data[(data['国家']=='China') & (data['排名']<=1000)].iterrows():
    cn_data_y.append([row['国际学生比例'], row['总分'], row['排名'], row['学校']])
    cn_data_x.append(row['国际教师比例'])
    
un_data_x, un_data_y = [], []
for idx, row in data[(data['国家']=='United States') & (data['排名']<=1000)].iterrows():
    un_data_y.append([row['国际学生比例'], row['总分'], row['排名'], row['学校']])
    un_data_x.append(row['国际教师比例'])
    
uk_data_x, uk_data_y = [], []
for idx, row in data[(data['国家']=='United Kingdom') & (data['排名']<=1000)].iterrows():
    uk_data_y.append([row['国际学生比例'], row['总分'], row['排名'], row['学校']])
    uk_data_x.append(row['国际教师比例'])

In [24]:
tool_js = """
function (obj) {
            var value = obj.value;
            var schema = [{name: '排名', index: 0, text: '总体排名'},
                            {name: '国际学生比例', index: 1, text: '国际学生'},
                            {name: '国际教师比例', index: 2, text: '国际教师'},
                            {name: '总分', index: 3, text: '总体分数'}];
            return '<div style="border-bottom: 1px solid rgba(255,255,255,.3); font-size: 18px;padding-bottom: 7px;margin-bottom: 7px">'
                + value[4] 
                + '</div>'
                + schema[0].text + '：' + value[3] + '<br>'
                + schema[1].text + '：' + value[1] + '<br>'
                + schema[2].text + '：' + value[0] + '<br>'
                + schema[3].text + '：' + value[2] + '<br>';
        }
"""

In [25]:
itemStyle_uk = {
    'opacity': 0.8,
    'shadowBlur': 10,
    'shadowOffsetX': 0,
    'shadowOffsetY': 0,
    'shadowColor': 'rgba(0, 0, 0, 0.5)',
    'color': '#80F1BE'
}

itemStyle_un = {
    'opacity': 0.8,
    'shadowBlur': 10,
    'shadowOffsetX': 0,
    'shadowOffsetY': 0,
    'shadowColor': 'rgba(0, 0, 0, 0.5)',
    'color': '#fec42c'
}

itemStyle_cn = {
    'opacity': 0.8,
    'shadowBlur': 10,
    'shadowOffsetX': 0,
    'shadowOffsetY': 0,
    'shadowColor': 'rgba(0, 0, 0, 0.5)',
    'color': '#dd4444'
}

In [26]:
visualMap = [{'left': 'right','top': '10%','dimension': 2,'min': 20,'max': 100,'itemWidth': 30,
              'itemHeight': 120,'calculable': True,'precision': 0.1,'text': ['圆形大小：总体分数'],'textGap': 30,
              'textStyle': {'color': '#fff'},'inRange': {'symbolSize': [10, 50]},
              'outOfRange': {'symbolSize': [10, 50],'color': ['rgba(255,255,255,.2)']},
              'controller': {'inRange': {'color': ['#c23531']},'outOfRange': {'color': ['#444']}}},
             {'left': 'right','bottom': '15%','dimension': 3,'min': 1,'max': 500,'itemHeight': 120,
              'precision': 0.1,'text': ['明暗：总体排名'],'textGap': 30,'textStyle': {'color': '#fff'},
              'inRange': {'colorLightness': [0.5, 1]},
              'outOfRange': {'color': ['rgba(255,255,255,.2)']},
              'controller': {'inRange': {'color': ['#c23531']},'outOfRange': {'color': ['#444']}}}]


In [27]:
scatter_cn = (Scatter(init_opts=opts.InitOpts(bg_color='#404a59',width='1000px', height='800px'))
            .add_xaxis(cn_data_x)
            .add_yaxis("中国", cn_data_y,
                       label_opts=opts.LabelOpts(is_show=False),
                       itemstyle_opts=itemStyle_cn
                       )
            .set_global_opts(yaxis_opts=opts.AxisOpts(name='国际学生', type_="value", is_scale=True, 
                                                      name_textstyle_opts={'color': '#fff', 'fontSize': 16},
                                                      axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#eee'))),
                             xaxis_opts=opts.AxisOpts(name='国际教师', type_="value", is_scale=True, 
                                                      name_textstyle_opts={'color': '#fff', 'fontSize': 16},
                                                      axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#eee'))),
                             tooltip_opts=opts.TooltipOpts(is_show=True, background_color='#222', border_color='#777', border_width=1, formatter=JsCode(tool_js)),
                             legend_opts=opts.LegendOpts(is_show=True, pos_right=10,
                                                         textstyle_opts=opts.TextStyleOpts(color='#fff',font_size=16)),
                             visualmap_opts=visualMap
                             )
            )

scatter_un = (Scatter(init_opts=opts.InitOpts(bg_color='#404a59',width='1000px', height='800px'))
            .add_xaxis(un_data_x)
            .add_yaxis("美国", un_data_y,
                       label_opts=opts.LabelOpts(is_show=False),
                       itemstyle_opts=itemStyle_un
                       )
            )
            
scatter_uk = (Scatter(init_opts=opts.InitOpts(bg_color='#404a59',width='1000px', height='800px'))
            .add_xaxis(uk_data_x)
            .add_yaxis("英国", uk_data_y,
                       label_opts=opts.LabelOpts(is_show=False),
                       itemstyle_opts=itemStyle_uk
                       )
            )
            
scatter_cn.overlap(scatter_un)
scatter_cn.overlap(scatter_uk)
scatter_cn.render_notebook()

图中：x轴为国际教师比例，y轴为国际学生比例，点的大小表示总体得分，点的明暗反映总排名。
1.可以看出在国际化程度上，英国高校表现十分突出，美国的整体分布情况则比较平均。
2.中国高校国际化程度相对较低，占据前列的基本上都是中国香港特别行政区的高校。

In [28]:
top_100_data = data[(data['loc'] == 'China (Mainland)') & (data['排名'] <= 100)]

data_cn = []
for idx, row in top_100_data.iterrows():
    data_cn.append([row['学术声誉'], row['雇主声誉'], row['师生比例'],
                    row['每个学院的引文'], row['国际教师比例'], row['国际学生比例'], row['学校']])


top_100_data = data[(data['loc'] == 'United States') & (data['排名'] <= 100)]

data_un = []
for idx, row in top_100_data.iterrows():
    data_un.append([row['学术声誉'], row['雇主声誉'], row['师生比例'],
                    row['每个学院的引文'], row['国际教师比例'], row['国际学生比例'], row['学校']])
                    
t_data = data[(data['loc'] == 'United Kingdom') & (data['排名'] <= 100)]

data_uk = []
for idx, row in top_100_data.iterrows():
    data_uk.append([row['学术声誉'], row['雇主声誉'], row['师生比例'],
                    row['每个学院的引文'], row['国际教师比例'], row['国际学生比例'], row['学校']])

split_line_style = {'color': 
                    ['rgba(238, 197, 102, 0.1)', 'rgba(238, 197, 102, 0.2)','rgba(238, 197, 102, 0.4)', 
                     'rgba(238, 197, 102, 0.6)','rgba(238, 197, 102, 0.8)', 'rgba(238, 197, 102, 1)'][::-1]}

In [29]:
tool_js = """
function (obj) {
            var value = obj.value;
            var schema = [{name: '学术声誉', index: 1, text: '学术声誉'},
                            {name: '雇主声誉', index: 2, text: '雇主声誉'},
                            {name: '师生比例', index: 3, text: '师生比'},
                            {name: '每个学院的引文', index: 3, text: '教员引用率'},
                            {name: '国际教师比例', index: 2, text: '国际教师'},
                            {name: '国际学生比例', index: 3, text: '国际学生'}];
            return '<div style="border-bottom: 1px solid rgba(255,255,255,.3); font-size: 18px;padding-bottom: 7px;margin-bottom: 7px">'
                + value[6] 
                + '</div>'
                + schema[0].text + '：' + value[0] + '<br>'
                + schema[1].text + '：' + value[1] + '<br>'
                + schema[2].text + '：' + value[2] + '<br>'
                + schema[3].text + '：' + value[3] + '<br>'
                + schema[4].text + '：' + value[4] + '<br>'
                + schema[5].text + '：' + value[5] + '<br>';
        }
"""

In [30]:
radar = Radar(init_opts=opts.InitOpts(theme='dark', bg_color='#161627', height='800px', width='1000px'))
radar.add_schema(shape='circle',
                 textstyle_opts=opts.TextStyleOpts(color='rgb(238, 197, 102)'),
                 axisline_opt=opts.LineStyleOpts(is_show=True, color='rgba(238, 197, 102, 1)'),
                 splitline_opt=opts.SplitLineOpts(is_show=True, linestyle_opts=split_line_style),
                 schema=[opts.RadarIndicatorItem(name='学术声誉', max_=100),
                         opts.RadarIndicatorItem(name='雇主声誉', max_=100),
                         opts.RadarIndicatorItem(name='师生比', max_=100),
                         opts.RadarIndicatorItem(name="教员引用率", max_=100),
                         opts.RadarIndicatorItem(name="国际教师", max_=100),
                         opts.RadarIndicatorItem(name="国际学生", max_=100)])

radar.add('中国', data_cn, symbol='none', is_selected=True,
          label_opts=opts.LabelOpts(is_show=False),
          linestyle_opts=opts.LineStyleOpts(color='#dd4444', width=1, opacity=0.5),
          areastyle_opts=opts.AreaStyleOpts(color='#dd4444', opacity=0.05))

radar.add('美国', data_un, symbol='none', is_selected=False,
          label_opts=opts.LabelOpts(is_show=False),
          linestyle_opts=opts.LineStyleOpts(color='#fec42c', width=1, opacity=0.5),
          areastyle_opts=opts.AreaStyleOpts(color='#fec42c', opacity=0.05))

radar.add('英国', data_uk, symbol='none', is_selected=False,
          label_opts=opts.LabelOpts(is_show=False),
          linestyle_opts=opts.LineStyleOpts(color='cyan', width=1, opacity=0.5),
          areastyle_opts=opts.AreaStyleOpts(color='cyan',opacity=0.05))

radar.set_global_opts(legend_opts=opts.LegendOpts(is_show=True, selected_mode='multiple', pos_bottom=5),
                      tooltip_opts=opts.TooltipOpts(is_show=True, background_color='#222', 
                          border_color='#777', border_width=1, formatter=JsCode(tool_js)),
                      title_opts=opts.TitleOpts(title="中美英——TOP100大学对比", pos_left='center',
                                                title_textstyle_opts=opts.TextStyleOpts(font_size=20)))


radar.render_notebook()

通过雷达图将各项评分指标清晰的展示出来，也可以更加清晰的看到中国的高校(尤其是大陆地区的高校)在国际教师比例和国际学生比例两项得分较低，而英美两国高校得分较为均衡(六边形得分)。

In [31]:
# 各项指标排名

tool_js = """
<div style="border-bottom: 1px solid rgba(255,255,255,.3); font-size: 18px;padding-bottom: 7px;margin-bottom: 7px">
                 {} 
                 </div>
                 排名：{} <br>
                 国家地区：{} <br>
                 加权总分：{} <br>
                 国际学生：{} <br>
                 国际教师：{} <br>
                 师生比例：{} <br>
                 学术声誉：{} <br>
                 雇主声誉：{} <br>
                 教员引用率：{} <br>
"""

In [32]:
top_1000_data = data[data['排名']<=1000]
item_rank = {
    '学术声誉':'学术声誉',
    '教员引用率':'每个学院的引文',
    '师生比':'师生比例',
    '雇主声誉':'雇主声誉',
    '国际教师':'国际教师比例',
    '国际学生':'国际学生比例'
}

In [33]:
tab = Tab()
for item in item_rank.items():
    top_1000_data = top_1000_data[top_1000_data[item[1]]> 0]
    top_1000_data = top_1000_data.sort_values(by=item[1] , ascending=True) 
    university, score = [], []
    for idx, row in top_1000_data.tail(100).iterrows():
        tjs = tool_js.format(row['学校'], row['排名'], row['国家'],row['总分'], row['国际学生比例'],
                             row['国际教师比例'],row['师生比例'],row['学术声誉'],row['雇主声誉'],
                             row['每个学院的引文'])
        if row['国家'] == 'China':
            university.append('🇨🇳 {}'.format(re.sub('（.*?）', '',row['学校'])))
        else:
            university.append(re.sub('（.*?）', '',row['学校']))
        score.append(opts.BarItem(name='', value=row[item[1]], tooltip_opts=opts.TooltipOpts(formatter=tjs)))
    bar = (Bar()
           .add_xaxis(university)
           .add_yaxis('', score, category_gap='30%')
           .set_global_opts(title_opts=opts.TitleOpts(title="2022年世界大学「{}」排名（QS） TOP 100".format(item[0]),
                                                      pos_left="center",
                                                      title_textstyle_opts=opts.TextStyleOpts(font_size=20)),
                            datazoom_opts=opts.DataZoomOpts(range_start=70, range_end=100, orient='vertical'),
                            visualmap_opts=opts.VisualMapOpts(is_show=False, max_=100, min_=90, dimension=0,
                                    range_color=['#00FFFF', '#FF7F50']),
                            legend_opts=opts.LegendOpts(is_show=False),
                            xaxis_opts=opts.AxisOpts(is_show=False, max_=100, is_scale=True),
                            yaxis_opts=opts.AxisOpts(axistick_opts=opts.AxisTickOpts(is_show=False),
                                                     axisline_opts=opts.AxisLineOpts(is_show=False),
                                                     axislabel_opts=opts.LabelOpts(font_size=12)))
           .set_series_opts(label_opts=opts.LabelOpts(is_show=True,
                                                      position='right',
                                                      font_style='italic'),
                            itemstyle_opts={"normal": 
                                            {"barBorderRadius": [30, 30, 30, 30],'shadowBlur': 10,
                                             'shadowColor': 'rgba(120, 36, 50, 0.5)','shadowOffsetY': 5,}}
    ).reversal_axis())
    
    grid = (
            Grid(init_opts=opts.InitOpts(theme='purple-passion', width='1000px', height='1200px'))
            .add(bar, grid_opts=opts.GridOpts(pos_right='10%', pos_left='20%'))
        )
    tab.add(grid, item[0])
    
tab.render_notebook()

最后我们将范围扩大，看看前1000的高校在各个评分标准下的具体得分情况。